# Final Lab Classification - Fashion MNIST

## Wince Larcen M. Rivano
## CS190-2P - CIS303

## Importing of Modules

In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Importing of Dataset

In [ ]:
X_train = pd.read_csv("X_fashion_mnist_train.csv")
X_test = pd.read_csv("X_fashion_mnist_test.csv")
y_train = pd.read_csv("y_fashion_mnist_train.csv")

In [ ]:
X_train.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,119,114,130,76,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,22,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,33,96,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,2,3,0,3,174,189,67,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,164,58,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,21,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,2,0,1,1,0,0,0,...,71,12,0,0,0,0,0,0,0,0


In [ ]:
y_train.head()

,label
0,9
1,0
2,0
3,3
4,0


In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (60000, 784)
X_test shape: (10000, 784)
y_train shape: (60000, 1)


In [ ]:
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum(), y_train.isnull().sum().sum())

0 0 0


## Feature Engineering and Scaling

In [ ]:
s_scaler = StandardScaler()
m_scaler = MinMaxScaler()

X_train_s_scaled = s_scaler.fit_transform(X_train)
X_test_s_scaled = s_scaler.transform(X_test)

X_train_m_scaled = m_scaler.fit_transform(X_train)
X_test_m_scaled = m_scaler.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA

s_pca = PCA(n_components=100, random_state=42)
X_train_s_pca = s_pca.fit_transform(X_train_s_scaled)
X_test_s_pca = s_pca.transform(X_test_s_scaled)

m_pca = PCA(n_components=100, random_state=42)
X_train_m_pca = m_pca.fit_transform(X_train_m_scaled)
X_test_m_pca = m_pca.transform(X_test_m_scaled)

raw_pca = PCA(n_components=100, random_state=42)
X_train_raw_pca = raw_pca.fit_transform(X_train)
X_test_raw_pca = raw_pca.transform(X_test)


In [ ]:
X_train_s_final, X_val_s, y_train_s_final, y_val_s = train_test_split(
    X_train_s_pca, y_train, test_size=0.2, random_state=42, stratify=y_train
)

X_train_m_final, X_val_m, y_train_m_final, y_val_m = train_test_split(
    X_train_m_pca, y_train, test_size=0.2, random_state=42, stratify=y_train
)

X_train_raw_final, X_val_raw, y_train_raw_final, y_val_raw = train_test_split(
    X_train_raw_pca, y_train, test_size=0.2, random_state=42, stratify=y_train
)

X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_s_scaled, y_train, test_size=0.2, stratify=y_train, random_state=42
)

## Neural Net Approach (Classification)

### MLP Approach

In [ ]:
num_classes = len(np.unique(y_train_final))
print(num_classes)

10


In [ ]:
import numpy as np

class F1Callback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_X, val_y = self.validation_data
        val_pred_probs = self.model.predict(val_X)
        val_pred = np.argmax(val_pred_probs, axis=1)

        f1 = f1_score(val_y, val_pred, average='macro')
        print(f" — val_f1: {f1:.4f}")

f1_cb = F1Callback((X_val, y_val))

In [ ]:
model = Sequential([
    Input(shape=(784,)),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(patience=5, factor=0.5)
]

model.fit(X_train_final, y_train_final,
          validation_data=(X_val, y_val),
          epochs=50, batch_size=64,
          callbacks=[f1_cb])


Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 — val_f1: 0.8594
750/750 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.7550 - loss: 0.7193 - val_accuracy: 0.8601 - val_loss: 0.3860
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
 — val_f1: 0.8701
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8415 - loss: 0.4328 - val_accuracy: 0.8715 - val_loss: 0.3545
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 — val_f1: 0.8804
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8568 - loss: 0.3834 - val_accuracy: 0.8804 - val_loss: 0.3250
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 — val_f1: 0.8802
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8655 - loss: 0.3583 - val_accuracy: 0.8812 - val_loss: 0.3240
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 — val_f1: 0.8785
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8706 - loss: 0.3455 - val_accuracy: 0.8787 - val_loss: 0.3301
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
 — val_f1: 0

In [ ]:
y_val_pred = np.argmax(model.predict(X_val), axis=1)
print("Final Val F1 Score:", f1_score(y_val, y_val_pred, average='macro'))

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Final Val F1 Score: 0.9071686476842737


### CNN Approach

In [ ]:
X_train = X_train.to_numpy().reshape(-1, 28, 28, 1).astype("float32") / 255.0
X_test = X_test.to_numpy().reshape(-1, 28, 28, 1).astype("float32") / 255.0

y_train = y_train.to_numpy().squeeze()


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)


In [ ]:
X_train_c_final, X_val_c, y_train_c_final, y_val_c = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [ ]:
print(X_train_c_final.shape)
print(X_val_c.shape)
print(y_train_c_final.shape)
print(y_val_c.shape)

(54000, 28, 28, 1)
(6000, 28, 28, 1)
(54000,)
(6000,)


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train_c_final)


In [ ]:
num_classes = 10

model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
class F1Callback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_X, val_y = self.validation_data
        val_pred_probs = self.model.predict(val_X, verbose=0)
        val_pred = np.argmax(val_pred_probs, axis=1)
        f1 = f1_score(val_y, val_pred, average='macro')
        print(f" — val_f1: {f1:.4f}")

f1_cb = F1Callback((X_val_c, y_val_c))

In [ ]:
model.fit(
    X_train_c_final, y_train_c_final,
    validation_data=(X_val_c, y_val_c),
    batch_size=64,
    epochs=30,
    callbacks=[f1_cb]
)

Epoch 1/30
842/844 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9155 - loss: 0.2217 — val_f1: 0.9195
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9156 - loss: 0.2217 - val_accuracy: 0.9187 - val_loss: 0.2148
Epoch 2/30
842/844 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9278 - loss: 0.1945 — val_f1: 0.9115
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9278 - loss: 0.1945 - val_accuracy: 0.9113 - val_loss: 0.2402
Epoch 3/30
842/844 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9371 - loss: 0.1703 — val_f1: 0.9075
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9371 - loss: 0.1703 - val_accuracy: 0.9067 - val_loss: 0.2629
Epoch 4/30
841/844 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9397 - loss: 0.1594 — val_f1: 0.9288
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9397 - loss: 0.1594 - val_accuracy: 0.9295 - val_loss: 0.1927
Epoch 5/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9490 - loss: 0.1388 — val_f1: 0.9243
844/844 ━━━━━━━━━

In [ ]:
y_val_pred_c = np.argmax(model.predict(X_val_c), axis=1)
print("Final Val F1 Score:", f1_score(y_val_c, y_val_pred_c, average='macro'))
print("Val Accuracy:", accuracy_score(y_val_c, y_val_pred_c ))

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Final Val F1 Score: 0.9273153408376784
Val Accuracy: 0.9276666666666666


In [ ]:
df_preds = pd.DataFrame({
    'predictions': y_test_pred_c
})

df_preds.to_csv('Rivano_FinalLabClassificationResults_CNN.csv', index=False)